#### This code will develop a Retrieval Augemented Generation (RAG) based chat bot using LangChain, PineCone and OpenAI with the following salient points
- Read the question/answer pairs of a company
- Convert each question/answer pair into a separate text chunk
- Convert the chunked question/answer pair into embeddings using OpenAI Embeddings
- Create an Index in Pinecone and upsert the embedded chunks in the Pinecone vector
- Take the query from user, vectorize it and then find the matching text chunk from Pinecone vector store
- Include the matching text chunk into the query which is to be sent to LLM to answer the specific question about the user.

In [1]:
import json
import time
import hashlib
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

/data/mn27889/miniconda3/envs/rag-chatbot/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Setting the Vector Database into the PineCone

#### Reading the API KEYS for OpenAI and Pinecone

In [2]:
creds_file = "../credentials.json"
    
with open(creds_file, 'r') as file:
    creds_data = json.load(file)
    openai_api_key = creds_data['OPENAI_API_KEY']
    pinecone_api_key = creds_data['PINECONE_API_KEY']

assert openai_api_key != None, ""
assert pinecone_api_key != None, ""

#### Loading the data from the text file

In [3]:
qa_data = pd.read_excel('AI.xlsx')

In [4]:
question_list = []
answer_list = []
answer_combined = ""

for index, row in qa_data.iterrows():
    question = row['QUESTION']
    answer = row['RESPONSE']
    
    if pd.isna(question) == False:
        question_list.append(str(question))
        answer_list.append(answer_combined)
        if str(answer) != "":
            answer_combined = str(answer)
    else:
        if str(answer) != "":
            answer_combined += " " + str(answer)
    
    # Handle the final row
    if index == len(qa_data) - 1:
        answer_list.append(answer_combined)

# Omit the first row in `answer_list` sicne that is needed
answer_list = answer_list[1:]

#### Dividing the question/answer data into chunks with each chunk representing a question/answer pair

In [5]:
chunked_text = []
for i in range(len(question_list)):
    qa_pair = 'QUESTION: ' + question_list[i] + '\n' + 'RESPONSE: ' + answer_list[i]
    chunked_text.append(qa_pair)

In [7]:
print(chunked_text[0])

QUESTION: Do I have acne?
RESPONSE: Acne is a common skin condition that affects most people at some stage in their lives. You are most likely to get acne on your face, back, and chest.


#### Defining the embeddings model to create the embeddings

In [8]:
embed_model = OpenAIEmbeddings(api_key=openai_api_key, model="text-embedding-ada-002")
chunked_embeddings = embed_model.embed_documents(chunked_text)

### Combine the embeddings and Chunked Text

In [9]:
def generate_short_id(content: str) -> str:
    """
    Generate a short ID based on the content using SHA-256 hash.

    Args:
    - content (str): The content for which the ID is generated.

    Returns:
    - short_id (str): The generated short ID.
    """
    hash_obj = hashlib.sha256()
    hash_obj.update(content.encode("utf-8"))
    return hash_obj.hexdigest()

In [10]:
data_with_metadata = []

for doc_text, embedding in zip(chunked_text, chunked_embeddings):
    doc_id = generate_short_id(doc_text)
    
    data_item = {
        "id": doc_id,
        "values": embedding,
        "metadata": {'text':doc_text} #include the text as metadata
    }
    
    data_with_metadata.append(data_item)

#### Creating the PineCone Index

In [11]:
# configure client
pc = Pinecone(api_key=pinecone_api_key)

# configure serverless spec
spec = ServerlessSpec(cloud='aws', region='us-east-1')

pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'rag-chatbot-description-ott2zv7.svc.aped-4627-b74a.pinecone.io',
              'metric': 'dotproduct',
              'name': 'rag-chatbot-description',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [12]:
# check for and delete index if already exists
index_name = 'rag-chatbot-qa'
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# we create a new index
pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='dotproduct',
        spec=spec
    )

# Wait until the index is ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1) 

#### Connecting to Index and Upserting our knowledgebase

In [13]:
# Connect to index
index = pc.Index(index_name)
time.sleep(1)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Upserting our dataset into the Pinecone index

In [14]:
index.upsert(vectors=data_with_metadata)

{'upserted_count': 143}

In [17]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 143}},
 'total_vector_count': 143}

# Answering queries using information present in the database

In [18]:
text_field = "text" # it represents the field with which the source text is present in the pinecone index
vectorstore = PineconeVectorStore(index, embed_model, text_field)

In [22]:
# query = "Tell me symptoms about allergic reaction?"
# vectorstore.similarity_search(query, k=3)

[Document(page_content='QUESTION: how do I know if I am having an allergic reaction?\nRESPONSE: Allergic reactions usually happen within a few minutes of exposure to an allergen. They can cause: sneezing a runny or blocked nose red, itchy, watery eyes wheezing and coughing a red, itchy rash worsening of asthma or eczema symptoms Most allergic reactions are mild. Occasionally a severe reaction called anaphylaxis or anaphylactic shock can happen. This is a medical emergency and needs urgent treatment.'),
 Document(page_content="QUESTION: What are the sign for allergic rhinitis?  \nRESPONSE: Allergic rhinitis causes cold-like symptoms. These include sneezing, itchiness and a blocked or runny nose. Symptoms usually start soon after you're exposed to an allergen. Some people only get allergic rhinitis for a few months at a time. This is because they're sensitive to seasonal allergens, such as tree or grass pollen. Other people get allergic rhinitis all year round."),
 Document(page_content=

In [23]:
# completion llm
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

To only answer the question to each query directly without remembring the past information, use `RetrievalQA.from_chain_type`

In [24]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

query = "Tell me something about allergy?"
print(qa.run(query))

/data/mn27889/miniconda3/envs/rag-chatbot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


An allergy is a reaction the body has to a substance. Allergies are very common and can affect more than 1 in 4 people in Europe at some point in their lives. They can be caused by various allergens like pollen, dust, certain foods, insect bites, and more. Allergic reactions can range from mild symptoms like sneezing and itching to severe reactions like anaphylaxis, which is a medical emergency.


To remember the information about the past conversation between the user and the robot, implement `ConversationalRetrievalChain.from_llm` with `chat_history` as the memory buffer. Please remember to use this if it is required since this will lead to more usage of tokens

In [25]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [27]:
query = "Can you tell me about Bird Flu?"
answer = conversation_chain.run(query)
print(answer)

Bird flu is a type of flu infection that spreads among birds, also known as avian influenza. It is rare for bird flu to spread to humans, and cases of bird flu in people are very rare. The main way to get bird flu is through close contact with infected birds, such as touching them or their droppings. Symptoms in birds may include sudden death, swollen head, closed and runny eyes, loss of appetite, difficulty breathing, diarrhea, and fewer eggs laid. Treatment for bird flu involves staying at home or being treated in the hospital, where antiviral medicines may be given to reduce the severity of the condition and prevent complications. To reduce the risk of bird flu, it is important to wash hands frequently, cook meat thoroughly, avoid contact with live birds, and not consume raw poultry or eggs.


In [28]:
query = "Please tell me more about this disease"
answer = conversation_chain.run(query)
print(answer)

The main signs that a bird may have bird flu include sudden death, swollen head, closed and runny eyes, loss of appetite, difficulty breathing, diarrhea, fewer eggs laid, or eggs with watery whites. 

As for treatment, in birds, there are no specific treatments for bird flu. Infected birds are usually culled to prevent the spread of the virus to other birds or humans. It is important to report any suspected cases of bird flu to the relevant authorities for proper management and control.
